In [23]:
import string

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(device)

True
1
cuda


In [31]:
EOS = "/s"
PAD = "pad"
UNK = "unk"

CHARS: str = string.printable
CHAR_TOKENS: list[str] = list(CHARS) + [EOS, PAD, UNK]
NGRAM_SIZE: int = 3
HIDDEN_SIZE: int = 768

num_chars = len(CHAR_TOKENS)
char_to_idx = {c: i for i, c in enumerate(CHAR_TOKENS)}

In [ ]:
# An Embedding for each slot in the the ngram, (e.g. 1, 2, 3 for a NGRAM_SIZE=3).
ngram_char_embeddings = [
    torch.nn.Embedding(num_embeddings=num_chars, embedding_dim=HIDDEN_SIZE)
    for _ in range(NGRAM_SIZE)
]

In [44]:
seq = "This is a test sentence. "
# Pad sequence if not divisble by NGRAM_SIZE.
seq = list(seq) + (NGRAM_SIZE - len(seq) % NGRAM_SIZE) * [PAD]
result = []
for ngram_slot_idx in range(NGRAM_SIZE):
    ngram_slot_chars = seq[ngram_slot_idx : len(seq) : NGRAM_SIZE]
    ngram_slot_char_idxs = torch.tensor([char_to_idx[c] for c in ngram_slot_chars])
    ngram_slot_embeddings = ngram_char_embeddings[ngram_slot_idx](ngram_slot_char_idxs)
    result.append(ngram_slot_embeddings)
result = torch.stack(result).sum(dim=0)
print(result.shape)

torch.Size([9, 768])


In [66]:
from transformers import RobertaConfig, RobertaForMaskedLM

model = RobertaForMaskedLM(
    config=RobertaConfig(
        vocab_size=2,  # won't use
        hidden_size=HIDDEN_SIZE,  # default 768
        max_position_embeddings=514,
        num_attention_heads=12,
        num_hidden_layers=6,
        type_vocab_size=1,
        attention_probs_dropout_prob=0,
        hidden_dropout_prob=0,
    )
)

In [67]:
cn_embeddings = torch.rand([1, 5, HIDDEN_SIZE])
lm_embeddings = model.roberta.forward(inputs_embeds=cn_embeddings).last_hidden_state
# then decode the lm embeddings back to chars